# Complete FastAPI Notes

## 📚 PART 1: THEORETICAL CONCEPTS

### 1. What is FastAPI?
FastAPI is a modern, high-performance web framework for building APIs with Python 3.7+ based on standard Python type hints. It's designed to be fast (comparable to NodeJS and Go), easy to code, and reduces about 40% of developer-induced errors. It automatically generates interactive API documentation and validates request/response data using Pydantic models.

### 2. ASGI (Asynchronous Server Gateway Interface)
ASGI is the spiritual successor to WSGI, designed to handle asynchronous Python web applications. FastAPI is built on top of Starlette (an ASGI framework), which allows it to handle async/await patterns efficiently. This makes FastAPI capable of handling thousands of concurrent connections, WebSockets, and long-running requests without blocking, making it ideal for modern real-time applications.

### 3. REST API Principles
REST (Representational State Transfer) is an architectural style where resources are identified by URLs and manipulated using standard HTTP methods: GET (retrieve), POST (create), PUT/PATCH (update), and DELETE (remove). RESTful APIs are stateless, meaning each request contains all necessary information. FastAPI makes building REST APIs intuitive by mapping Python functions to these HTTP methods with simple decorators.

### 4. Path Operations & Routing
Path operations define how your API responds to different endpoints and HTTP methods. In FastAPI, the `@app.get()`, `@app.post()`, etc. decorators bind URL paths to Python functions. The framework uses path parameters `{item_id}` for dynamic URLs and query parameters for filtering/pagination. FastAPI automatically validates these parameters based on type hints, reducing boilerplate code significantly.

### 5. Dependency Injection System
FastAPI's dependency injection allows you to declare reusable logic (database connections, authentication, pagination) that gets automatically executed before your path operation. Dependencies can depend on other dependencies, creating a powerful tree of reusable components. This promotes clean code architecture, makes testing easier, and eliminates repetitive code across multiple endpoints.

### 6. Request & Response Lifecycle
When a request hits FastAPI: (1) The URL is matched to a path operation, (2) Path/query parameters are extracted and validated, (3) Request body is parsed and validated against Pydantic models, (4) Dependencies are resolved and executed, (5) Your function runs, (6) The response is validated and serialized, (7) Documentation is auto-updated. This entire pipeline provides type safety and automatic validation at every step.

### 7. Automatic API Documentation
FastAPI automatically generates interactive API documentation using OpenAPI standards. It creates two UIs: Swagger UI (at `/docs`) and ReDoc (at `/redoc`). These interfaces allow you to test endpoints directly from your browser, see request/response schemas, and understand your API without reading code. The documentation updates automatically as you modify your code.

### 8. Type Hints & Validation
Python type hints (`: str`, `: int`, etc.) aren't just for documentation in FastAPI—they drive the entire validation and serialization system. FastAPI uses these hints with Pydantic to validate incoming data, convert types, and generate error messages. This means you catch errors before they reach your business logic, and your code becomes self-documenting and IDE-friendly.

---

## 💻 PART 2: CODE EXAMPLES (LEVEL 1 - BEGINNER)

### Basic Application Setup
```python
from fastapi import FastAPI

# Create FastAPI application instance
app = FastAPI(
    title="My API",
    description="A comprehensive FastAPI example",
    version="1.0.0"
)

# Simple GET endpoint
@app.get("/")
def read_root():
    """Root endpoint returning a welcome message"""
    return {"message": "Welcome to FastAPI!"}

# GET with path parameter
@app.get("/items/{item_id}")
def read_item(item_id: int):
    """Retrieve item by ID"""
    return {"item_id": item_id, "name": f"Item {item_id}"}

# GET with query parameters
@app.get("/search")
def search_items(query: str, limit: int = 10):
    """Search items with pagination"""
    return {
        "query": query,
        "limit": limit,
        "results": [f"Result {i}" for i in range(limit)]
    }
```

### Basic Pydantic Models
```python
from pydantic import BaseModel, Field
from typing import Optional

# Simple Pydantic model for request validation
class Item(BaseModel):
    name: str
    description: Optional[str] = None
    price: float
    tax: Optional[float] = None

# POST endpoint with request body
@app.post("/items")
def create_item(item: Item):
    """Create a new item"""
    total_price = item.price + (item.tax if item.tax else 0)
    return {"item": item, "total_price": total_price}

# Model with validation rules
class User(BaseModel):
    username: str = Field(..., min_length=3, max_length=50)
    email: str = Field(..., regex=r"^[\w\.-]+@[\w\.-]+\.\w+$")
    age: int = Field(..., gt=0, le=120)
    is_active: bool = True

@app.post("/users")
def create_user(user: User):
    """Create user with validated data"""
    return {"user_created": user.username, "email": user.email}
```

### HTTP Methods (CRUD Operations)
```python
# In-memory database simulation
fake_db = {}

# CREATE - POST
@app.post("/products/{product_id}")
def create_product(product_id: int, item: Item):
    """Create a new product"""
    fake_db[product_id] = item
    return {"message": "Product created", "product": item}

# READ - GET
@app.get("/products/{product_id}")
def get_product(product_id: int):
    """Get product by ID"""
    if product_id not in fake_db:
        return {"error": "Product not found"}
    return fake_db[product_id]

# UPDATE - PUT (full update)
@app.put("/products/{product_id}")
def update_product(product_id: int, item: Item):
    """Update entire product"""
    fake_db[product_id] = item
    return {"message": "Product updated", "product": item}

# PARTIAL UPDATE - PATCH
@app.patch("/products/{product_id}")
def partial_update(product_id: int, name: Optional[str] = None, price: Optional[float] = None):
    """Partially update product"""
    if product_id in fake_db:
        if name:
            fake_db[product_id].name = name
        if price:
            fake_db[product_id].price = price
        return {"message": "Product updated", "product": fake_db[product_id]}
    return {"error": "Product not found"}

# DELETE
@app.delete("/products/{product_id}")
def delete_product(product_id: int):
    """Delete a product"""
    if product_id in fake_db:
        del fake_db[product_id]
        return {"message": "Product deleted"}
    return {"error": "Product not found"}
```

---

## 🚀 PART 2: CODE EXAMPLES (LEVEL 2 - INTERMEDIATE)

### Response Models & Status Codes
```python
from fastapi import status, HTTPException
from typing import List

# Response model (what API returns)
class ItemResponse(BaseModel):
    id: int
    name: str
    price: float
    in_stock: bool

# Using response_model to control output
@app.get("/items/{item_id}", response_model=ItemResponse, status_code=status.HTTP_200_OK)
def get_item_detailed(item_id: int):
    """Returns only fields defined in ItemResponse"""
    # Imagine we fetch from DB with sensitive data
    item_data = {
        "id": item_id,
        "name": "Laptop",
        "price": 999.99,
        "in_stock": True,
        "internal_code": "SECRET123",  # This won't be returned
        "supplier_cost": 500.00  # This won't be returned
    }
    return item_data

# Return list of items
@app.get("/items", response_model=List[ItemResponse])
def list_items(skip: int = 0, limit: int = 10):
    """Get paginated list of items"""
    items = [
        {"id": i, "name": f"Item {i}", "price": i * 10.0, "in_stock": i % 2 == 0}
        for i in range(skip, skip + limit)
    ]
    return items

# Custom status codes
@app.post("/items", status_code=status.HTTP_201_CREATED)
def create_item_with_status(item: Item):
    """Returns 201 Created status"""
    return {"created": True, "item": item}
```

### Exception Handling
```python
from fastapi import HTTPException

# Custom exception example
@app.get("/users/{user_id}")
def get_user(user_id: int):
    """Get user with proper error handling"""
    if user_id not in range(1, 100):
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"User {user_id} not found",
            headers={"X-Error": "UserNotFound"}
        )
    return {"user_id": user_id, "username": f"user_{user_id}"}

# Multiple validation checks
@app.post("/orders")
def create_order(item_id: int, quantity: int):
    """Create order with business logic validation"""
    if quantity <= 0:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Quantity must be positive"
        )
    if quantity > 100:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Cannot order more than 100 items at once"
        )
    return {"order_created": True, "item_id": item_id, "quantity": quantity}
```

### Dependency Injection Basics
```python
from fastapi import Depends

# Simple dependency function
def get_query_token(token: str):
    """Dependency to validate token"""
    if token != "secret-token":
        raise HTTPException(status_code=400, detail="Invalid token")
    return token

# Using dependency
@app.get("/protected", dependencies=[Depends(get_query_token)])
def protected_route():
    """Route protected by token dependency"""
    return {"message": "You have access!"}

# Dependency that returns value
def get_current_user(username: str):
    """Simulated user authentication"""
    if username == "admin":
        return {"username": username, "role": "admin"}
    return {"username": username, "role": "user"}

@app.get("/dashboard")
def dashboard(user: dict = Depends(get_current_user)):
    """Dashboard with user dependency"""
    return {
        "message": f"Welcome {user['username']}",
        "role": user["role"]
    }

# Class-based dependency
class Pagination:
    def __init__(self, skip: int = 0, limit: int = 10):
        self.skip = skip
        self.limit = limit

@app.get("/posts")
def get_posts(pagination: Pagination = Depends()):
    """Using class-based dependency for pagination"""
    return {
        "skip": pagination.skip,
        "limit": pagination.limit,
        "posts": [f"Post {i}" for i in range(pagination.skip, pagination.skip + pagination.limit)]
    }
```

### Background Tasks
```python
from fastapi import BackgroundTasks

# Background task function
def write_log(message: str):
    """Simulated log writing"""
    with open("log.txt", "a") as log:
        log.write(f"{message}\n")

def send_email(email: str, message: str):
    """Simulated email sending"""
    print(f"Sending email to {email}: {message}")

# Endpoint with background task
@app.post("/send-notification")
def send_notification(email: str, background_tasks: BackgroundTasks):
    """Send notification and log in background"""
    background_tasks.add_task(send_email, email, "Your order is confirmed")
    background_tasks.add_task(write_log, f"Email sent to {email}")
    return {"message": "Notification will be sent in background"}
```

---

## ⚡ PART 2: CODE EXAMPLES (LEVEL 3 - ADVANCED)

### Advanced Pydantic Models
```python
from pydantic import BaseModel, validator, root_validator, Field
from typing import List, Optional, Union
from datetime import datetime
from enum import Enum

# Enum for choices
class Category(str, Enum):
    ELECTRONICS = "electronics"
    CLOTHING = "clothing"
    FOOD = "food"

# Nested models
class Address(BaseModel):
    street: str
    city: str
    country: str
    zip_code: str = Field(..., regex=r"^\d{5}(-\d{4})?$")

class User(BaseModel):
    username: str
    email: str
    addresses: List[Address] = []
    created_at: datetime = Field(default_factory=datetime.now)

# Custom validators
class Product(BaseModel):
    name: str
    price: float
    category: Category
    tags: List[str] = []
    
    @validator('price')
    def price_must_be_positive(cls, v):
        """Validate price is positive"""
        if v <= 0:
            raise ValueError('Price must be positive')
        return v
    
    @validator('name')
    def name_must_not_be_empty(cls, v):
        """Validate name is not empty"""
        if not v.strip():
            raise ValueError('Name cannot be empty')
        return v.strip()
    
    @root_validator
    def check_electronics_price(cls, values):
        """Cross-field validation"""
        category = values.get('category')
        price = values.get('price')
        if category == Category.ELECTRONICS and price and price < 10:
            raise ValueError('Electronics must cost at least $10')
        return values

# Model configuration
class ItemAdvanced(BaseModel):
    name: str
    price: float
    
    class Config:
        schema_extra = {
            "example": {
                "name": "Laptop",
                "price": 999.99
            }
        }

@app.post("/advanced-products")
def create_advanced_product(product: Product):
    """Create product with advanced validation"""
    return {"product": product, "valid": True}
```

### Request Headers & Cookies
```python
from fastapi import Header, Cookie

# Read headers
@app.get("/headers")
def read_headers(
    user_agent: Optional[str] = Header(None),
    accept_language: Optional[str] = Header(None),
    x_custom_header: Optional[str] = Header(None)
):
    """Read request headers"""
    return {
        "User-Agent": user_agent,
        "Accept-Language": accept_language,
        "X-Custom-Header": x_custom_header
    }

# Work with cookies
@app.get("/cookies")
def read_cookies(session_id: Optional[str] = Cookie(None)):
    """Read cookies from request"""
    return {"session_id": session_id}

from fastapi import Response

@app.post("/set-cookie")
def set_cookie(response: Response):
    """Set cookie in response"""
    response.set_cookie(
        key="session_id",
        value="abc123",
        max_age=3600,
        httponly=True
    )
    return {"message": "Cookie set"}
```

### File Upload & Download
```python
from fastapi import File, UploadFile
from fastapi.responses import FileResponse, StreamingResponse
import shutil
from pathlib import Path

# Single file upload
@app.post("/upload")
async def upload_file(file: UploadFile = File(...)):
    """Upload a single file"""
    contents = await file.read()
    file_size = len(contents)
    
    # Save file
    save_path = Path(f"uploads/{file.filename}")
    save_path.parent.mkdir(exist_ok=True)
    
    with open(save_path, "wb") as f:
        f.write(contents)
    
    return {
        "filename": file.filename,
        "content_type": file.content_type,
        "size": file_size
    }

# Multiple file upload
@app.post("/upload-multiple")
async def upload_multiple_files(files: List[UploadFile] = File(...)):
    """Upload multiple files"""
    uploaded = []
    for file in files:
        contents = await file.read()
        save_path = Path(f"uploads/{file.filename}")
        with open(save_path, "wb") as f:
            f.write(contents)
        uploaded.append(file.filename)
    
    return {"uploaded_files": uploaded}

# File download
@app.get("/download/{filename}")
def download_file(filename: str):
    """Download a file"""
    file_path = Path(f"uploads/{filename}")
    if not file_path.exists():
        raise HTTPException(status_code=404, detail="File not found")
    
    return FileResponse(
        path=file_path,
        filename=filename,
        media_type="application/octet-stream"
    )
```

### Middleware & CORS
```python
from fastapi.middleware.cors import CORSMiddleware
from fastapi.middleware.trustedhost import TrustedHostMiddleware
import time

# CORS configuration
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000", "https://example.com"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Custom middleware
@app.middleware("http")
async def add_process_time_header(request, call_next):
    """Add custom header with request processing time"""
    start_time = time.time()
    response = await call_next(request)
    process_time = time.time() - start_time
    response.headers["X-Process-Time"] = str(process_time)
    return response

# Trusted host middleware
app.add_middleware(
    TrustedHostMiddleware,
    allowed_hosts=["localhost", "*.example.com"]
)
```

### Async Operations & Database
```python
from typing import AsyncGenerator
import asyncio

# Simulated async database connection
class AsyncDatabase:
    async def connect(self):
        await asyncio.sleep(0.1)  # Simulate connection
        return self
    
    async def fetch_user(self, user_id: int):
        await asyncio.sleep(0.1)  # Simulate query
        return {"user_id": user_id, "name": f"User {user_id}"}
    
    async def close(self):
        await asyncio.sleep(0.1)  # Simulate cleanup

# Async dependency
async def get_db() -> AsyncGenerator:
    """Database connection dependency"""
    db = AsyncDatabase()
    await db.connect()
    try:
        yield db
    finally:
        await db.close()

# Async endpoint
@app.get("/async-users/{user_id}")
async def get_user_async(user_id: int, db: AsyncDatabase = Depends(get_db)):
    """Async endpoint with database"""
    user = await db.fetch_user(user_id)
    return user

# Multiple async operations
@app.get("/fetch-multiple")
async def fetch_multiple_resources():
    """Fetch multiple resources concurrently"""
    async def fetch_user(user_id):
        await asyncio.sleep(0.5)
        return {"user_id": user_id, "name": f"User {user_id}"}
    
    async def fetch_posts(user_id):
        await asyncio.sleep(0.5)
        return [{"post_id": i, "title": f"Post {i}"} for i in range(3)]
    
    # Run concurrently
    user, posts = await asyncio.gather(
        fetch_user(1),
        fetch_posts(1)
    )
    
    return {"user": user, "posts": posts}
```

### WebSocket Support
```python
from fastapi import WebSocket, WebSocketDisconnect
from typing import List

# WebSocket connection manager
class ConnectionManager:
    def __init__(self):
        self.active_connections: List[WebSocket] = []
    
    async def connect(self, websocket: WebSocket):
        await websocket.accept()
        self.active_connections.append(websocket)
    
    def disconnect(self, websocket: WebSocket):
        self.active_connections.remove(websocket)
    
    async def broadcast(self, message: str):
        for connection in self.active_connections:
            await connection.send_text(message)

manager = ConnectionManager()

@app.websocket("/ws/{client_id}")
async def websocket_endpoint(websocket: WebSocket, client_id: int):
    """WebSocket endpoint for real-time communication"""
    await manager.connect(websocket)
    try:
        while True:
            data = await websocket.receive_text()
            await manager.broadcast(f"Client {client_id}: {data}")
    except WebSocketDisconnect:
        manager.disconnect(websocket)
        await manager.broadcast(f"Client {client_id} left the chat")
```

---

## 📖 PART 3: PYDANTIC LIBRARY DEEP DIVE

### Core Pydantic Features

#### 1. Field Types & Validation
```python
from pydantic import (
    BaseModel, Field, EmailStr, HttpUrl, SecretStr, 
    conint, constr, confloat, condecimal
)
from typing import Optional, List, Dict, Set, Tuple
from datetime import datetime, date, time
from decimal import Decimal

class ComprehensiveModel(BaseModel):
    # String types
    name: str = Field(..., min_length=1, max_length=100)
    email: EmailStr  # Requires: pip install pydantic[email]
    website: HttpUrl
    password: SecretStr  # Never logged or displayed
    
    # Constrained types
    age: conint(gt=0, le=120)  # Constrained integer
    username: constr(regex=r'^[a-zA-Z0-9_]+$', min_length=3)
    rating: confloat(ge=0.0, le=5.0)  # Constrained float
    price: condecimal(max_digits=10, decimal_places=2)
    
    # Collections
    tags: List[str] = []
    metadata: Dict[str, str] = {}
    unique_tags: Set[str] = set()
    coordinates: Tuple[float, float]
    
    # Date and time
    created_at: datetime
    birth_date: date
    appointment_time: time
    
    # Optional fields
    description: Optional[str] = None
    score: Optional[float] = Field(None, ge=0, le=100)
```

#### 2. Advanced Validators
```python
from pydantic import BaseModel, validator, root_validator

class Order(BaseModel):
    product_id: int
    quantity: int
    unit_price: float
    discount: float = 0.0
    total: Optional[float] = None
    
    @validator('quantity')
    def quantity_must_be_positive(cls, v):
        """Field-level validator"""
        if v <= 0:
            raise ValueError('Quantity must be positive')
        return v
    
    @validator('discount')
    def discount_in_range(cls, v):
        """Validate discount percentage"""
        if not 0 <= v <= 100:
            raise ValueError('Discount must be between 0 and 100')
        return v
    
    @validator('unit_price', 'discount', pre=True)
    def convert_to_float(cls, v):
        """Pre-validator to convert strings"""
        if isinstance(v, str):
            return float(v.replace(',', '.'))
        return v
    
    @root_validator
    def calculate_total(cls, values):
        """Cross-field validation and calculation"""
        quantity = values.get('quantity')
        unit_price = values.get('unit_price')
        discount = values.get('discount', 0)
        
        if quantity and unit_price:
            subtotal = quantity * unit_price
            discount_amount = subtotal * (discount / 100)
            values['total'] = round(subtotal - discount_amount, 2)
        
        return values

# Usage
order = Order(product_id=1, quantity=5, unit_price=29.99, discount=10)
print(order.total)  # Automatically calculated: 134.96
```

#### 3. Model Configuration
```python
class ConfiguredModel(BaseModel):
    name: str
    internal_id: str
    
    class Config:
        # Allow fields not in model (useful for forward compatibility)
        extra = "forbid"  # or "allow" or "ignore"
        
        # Validate on assignment (not just initialization)
        validate_assignment = True
        
        # Use enum values instead of enum objects
        use_enum_values = True
        
        # Allow arbitrary types (for custom classes)
        arbitrary_types_allowed = True
        
        # JSON schema customization
        schema_extra = {
            "example": {
                "name": "Example Name",
                "internal_id": "ID123"
            }
        }
        
        # Field alias (API uses snake_case, Python uses camelCase)
        fields = {
            'internal_id': {'alias': 'internalId'}
        }
```

#### 4. Model Inheritance & Composition
```python
# Base model
class BaseItem(BaseModel):
    id: int
    created_at: datetime = Field(default_factory=datetime.now)

# Inherit and extend
class Product(BaseItem):
    name: str
    price: float
    
class Book(Product):
    author: str
    isbn: str
    pages: int

# Model composition
class Author(BaseModel):
    name: str
    bio: str

class BookWithAuthor(BaseModel):
    title: str
    author: Author  # Nested model
    price: float

# Usage
book = BookWithAuthor(
    title="FastAPI Guide",
    author={"name": "John Doe", "bio": "Developer"},
    price=29.99
)
```

#### 5. Serialization & Parsing
```python
from pydantic import BaseModel

class User(BaseModel):
    username: str
    email: str
    is_active: bool = True

# Create from dict
user_data = {"username": "john", "email": "john@example.com"}
user = User(**user_data)

# Convert to dict
user_dict = user.dict()
# {'username': 'john', 'email': 'john@example.com', 'is_active': True}

# Convert to dict excluding certain fields
user_dict_minimal = user.dict(exclude={'is_active'})

# Convert to dict including only certain fields
user_dict_selected = user.dict(include={'username', 'email'})

# Convert to JSON string
user_json = user.json()

# Parse from JSON string
json_str = '{"username": "jane", "email": "jane@example.com"}'
user_from_json = User.parse_raw(json_str)

# Copy and update
updated_user = user.copy(update={'username': 'john_updated'})
```

---

## 🎯 BEST PRACTICES & TIPS

### 1. Project Structure
```
my_fastapi_project/
├── app/
│   ├── __init__.py
│   ├── main.py              # FastAPI app instance
│   ├── models.py            # Pydantic models
│   ├── database.py          # Database connection
│   ├── dependencies.py      # Shared dependencies
│   └── routers/             # Route modules
│       ├── __init__.py
│       ├── users.py
│       └── items.py
├── tests/
├── requirements.txt
└── .env
```

### 2. Router Organization
```python
# routers/users.py
from fastapi import APIRouter

router = APIRouter(
    prefix="/users",
    tags=["users"],
    responses={404: {"description": "Not found"}}
)

@router.get("/")
def list_users():
    return {"users": []}

@router.get("/{user_id}")
def get_user(user_id: int):
    return {"user_id": user_id}

# main.py
from app.routers import users, items

app.include_router(users.router)
app.include_router(items.router)
```

### 3. Environment Variables
```python
from pydantic import BaseSettings

class Settings(BaseSettings):
    app_name: str = "FastAPI App"
    database_url: str
    secret_key: str
    debug: bool = False
    
    class Config:
        env_file = ".env"

settings = Settings()
```

### 4. Testing FastAPI
```python
from fastapi.testclient import TestClient

client = TestClient(app)

def test_read_root():
    response = client.get("/")
    assert response.status_code == 200
    assert response.json() == {"message": "Welcome to FastAPI!"}

def test_create_item():
    response = client.post(
        "/items",
        json={"name": "Test", "price": 10.5}
    )
    assert response.status_code == 201
```

---

## 🚀 RUNNING YOUR APPLICATION

### Development
```bash
# Install FastAPI and Uvicorn
pip install fastapi uvicorn[standard]

# Run with auto-reload
uvicorn main:app --reload --host 0.0.0.0 --port 8000

# Access interactive docs
# Swagger UI: http://localhost:8000/docs
# ReDoc: http://localhost:8000/redoc
```

### Production
```bash
# Run with multiple workers
uvicorn main:app --host 0.0.0.0 --port 8000 --workers 4

# Or use Gunicorn with Uvicorn workers
gunicorn main:app --workers 4 --worker-class uvicorn.workers.UvicornWorker --bind 0.0.0.0:8000
```

---

**End of Notes** 📝